# Pipeline

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# read the data
X = pd.read_csv('./train.csv', index_col='Id')
X_test = pd.read_csv('./test.csv', index_col='Id')

# remove rows with missing target
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

# separate target from predictors
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# split dataset into training and validation data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# categorical columns with low cardinality
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object' and X_train[col].nunique() < 10]

# numerical columns
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

# keep selected columns
selected_cols = categorical_cols + numerical_cols
selected_X_train = X_train[selected_cols].copy()
selected_X_valid = X_valid[selected_cols].copy()
selected_X_test = X_test[selected_cols].copy()

In [2]:
selected_X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


In [4]:
selected_X_train.isnull().sum()

MSZoning          0
Street            0
Alley          1097
LotShape          0
LandContour       0
               ... 
ScreenPorch       0
PoolArea          0
MiscVal           0
MoSold            0
YrSold            0
Length: 76, dtype: int64

### Simple Pipeline

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='constant')),
  ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[
  ('num', numerical_transformer, numerical_cols),
  ('cat', categorical_transformer, categorical_cols)
])

# define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# bundle preprocessor and model into a pipeline
pipeline = Pipeline(steps=[
  ('preprocessor', preprocessor),
  ('model', model)
])

# proprocessing training data -> fit model
pipeline.fit(selected_X_train, y_train)

# preprocessing validation data -> predict from model
preds = pipeline.predict(selected_X_valid)

print(f"MAE score: {mean_absolute_error(y_valid, preds)}")


MAE score: 17495.59904109589


### Generate test prediction

In [11]:
# preprocessing test data
preds_test = pipeline.predict(selected_X_test)

# save prediction into a file
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('pipeline_predictions.csv', index=False)

In [12]:
output.head()

,Id,SalePrice
0,1461,126370.00
1,1462,153450.50
2,1463,185135.80
3,1464,181598.85
4,1465,197399.00
